# Installs and Imports

## Installs

In [2]:
%pip install neo4j

     ---------------------------------------- 90.8/90.8 KB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 503.5/503.5 KB 7.8 MB/s eta 0:00:00
Using legacy 'setup.py install' for neo4j, since package 'wheel' is not installed.
  Running setup.py install for neo4j: started
  Running setup.py install for neo4j: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\nelso\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


## Imports

In [3]:
from neo4j import GraphDatabase, basic_auth
import mdb_tools.mdb_tools as mdb

# Database connection

URL, username, and password for database connection

In [4]:
# MDB sandbox
url = "bolt://localhost:7687" #"<URL for database>"
user = "neo4j" #"<Username for database>"
password = "noble-use-dairy" #"<Password for database>"
driver = GraphDatabase.driver(url, auth=(user, password))

In [12]:
dir(mdb)

['GraphDatabase',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'basic_auth',
 'check_term_exists',
 'create_concept',
 'create_object_relationship',
 'create_predicate',
 'create_represents_relationship',
 'create_represents_relationship_from_nanoid',
 'create_subject_relationship',
 'create_term',
 'create_term_from_nanoid',
 'detach_delete_concept',
 'detach_delete_predicate',
 'generate',
 'generate_unique_nanoid',
 'get_concept',
 'get_terms',
 'link_concepts_to_predicate',
 'link_two_terms',
 'make_nano',
 'merge_two_concepts']

# Linking Terms


### Check existence of Term

In [5]:
with driver.session() as session:
  term_exists = session.read_transaction(mdb.check_term_exists, "Epithelioma, benign")
  if term_exists:
    print("Term with that value found in DB")
  else:
    print("Term with that value not found in DB")

driver.close()

Term with that value found in DB


### Find existing Concept from Term

### Create new Term

In [7]:
with driver.session() as session:
    session.write_transaction(mdb.create_term, "Epithelioma, benign")
    session.write_transaction(mdb.create_term, "Epithelial tumor, benign")
    session.write_transaction(mdb.create_term, "Spindle cell carcinoma, NOS")
    session.write_transaction(mdb.create_term, "Sarcomatoid Carcinoma")
    session.write_transaction(mdb.create_term, "Minimally Invasive Lung Adenocarcinoma")
    session.write_transaction(mdb.create_term, "Undifferentiated Carcinoma")
driver.close()

Created new Term with value: Epithelioma, benign
Created new Term with value: Epithelial tumor, benign
Created new Term with value: Spindle cell carcinoma, NOS
Created new Term with value: Sarcomatoid Carcinoma
Created new Term with value: Minimally Invasive Lung Adenocarcinoma
Created new Term with value: Undifferentiated Carcinoma


### Generate unique Concept nanoid

In [81]:
with driver.session() as session:
  session.read_transaction(mdb.generate_unique_nanoid)
driver.close()

hjpBQV


### Create Concept

In [12]:
with driver.session() as session:
    id_1 = session.read_transaction(mdb.generate_unique_nanoid)
    session.write_transaction(mdb.create_concept, id_1)
    id_2 = session.read_transaction(mdb.generate_unique_nanoid)   
    session.write_transaction(mdb.create_concept, id_2)
driver.close()

Created new Concept with nanoid: H6X2Ph
Created new Concept with nanoid: p2IbFv


### Link Term and Concept

In [15]:
with driver.session() as session:
    session.write_transaction(mdb.create_represents_relationship, "Epithelioma, benign", id_1)
    session.write_transaction(mdb.create_represents_relationship, "Epithelial tumor, benign", id_1)
    session.write_transaction(mdb.create_represents_relationship, "Minimally Invasive Lung Adenocarcinoma", id_2)
driver.close() 

Epithelioma, benign Term now represents H6X2Ph Concept
Epithelial tumor, benign Term now represents H6X2Ph Concept
Minimally Invasive Lung Adenocarcinoma Term now represents p2IbFv Concept


## Link Two Terms

In [18]:
# both terms exist & connected via concept
mdb.link_two_terms("Epithelioma, benign", "Epithelial tumor, benign")

Epithelioma, benign and Epithelial tumor, benign are already connected via Concept H6X2Ph


In [19]:
# both terms exist & NOT connected via concept
mdb.link_two_terms("Spindle cell carcinoma, NOS", "Sarcomatoid Carcinoma")

Created new Concept with nanoid: Zith4U
Spindle cell carcinoma, NOS Term now represents Zith4U Concept
Sarcomatoid Carcinoma Term now represents Zith4U Concept


In [20]:
# one term exists & already has concept
mdb.link_two_terms("Minimally Invasive Lung Adenocarcinoma", "Alveolar adenocarcinoma")

Created new Term with value: Alveolar adenocarcinoma
Alveolar adenocarcinoma Term now represents p2IbFv Concept


In [21]:
# one term exists & doesn't have concept yet
mdb.link_two_terms("Undifferentiated Carcinoma", "Carcinoma, anaplastic, NOS")

Created new Concept with nanoid: dBSPWS
Created new Term with value: Carcinoma, anaplastic, NOS
Undifferentiated Carcinoma Term now represents dBSPWS Concept
Carcinoma, anaplastic, NOS Term now represents dBSPWS Concept


In [22]:
# neither term exists
mdb.link_two_terms("Epithelioma, malignant", "Carcinoma")

Created new Term with value: Epithelioma, malignant
Created new Term with value: Carcinoma
Created new Concept with nanoid: 4CneRp
Epithelioma, malignant Term now represents 4CneRp Concept
Carcinoma Term now represents 4CneRp Concept


# Linking Concepts


### Create Predicate

In [ ]:
with driver.session() as session:
    pred_nanoid_1 = session.read_transaction(mdb.generate_unique_nanoid)
    pred_handle_1 = "exactMatch"
    session.write_transaction(mdb.create_predicate, pred_nanoid_1, pred_handle_1)
driver.close() 

### Link two Concepts to a Predicate

General pattern: (c1:concept)<-[:has_subject]-(p:predicate {handle:“exactMatch”})-[:has_object]->(c2:concept)

Example: Concepts with nanoid: "4jtzA3" and nanoid: "UWs6Di" both represent terms with value: Lung in the MDB. Let's link them with the predicate we just created.

In [ ]:
with driver.session() as session:
    predicate_nanoid = "2cEzXW" #"<predicate_nanoid_from_above>"
    concept_nanoid_1 = "4jtzA3"
    concept_nanoid_2 = "UWs6Di"
    session.write_transaction(mdb.create_subject_relationship, 
                                concept_nanoid_1, predicate_nanoid)
    session.write_transaction(mdb.create_object_relationship, 
                                concept_nanoid_2, predicate_nanoid)
driver.close()

Created has_subject relationship between source Predicate: 2cEzXW and destination Concept: 4jtzA3
Created has_object relationship between source Predicate: 2cEzXW and destination Concept: UWs6Di


When two existing Concept nodes are deemed synonymous, there are two primary ways to approach linking them together. The first way to link the synonymous Concepts would be via a Predicate node with the an "exactMatch" handle. This method maintains the exisiting Concept & Term structure while adding to it, allowing queries already in use to continue to work. 

The second way is simply merging the two so they are represented by the same Concept node. With this approach, the Terms linked to each Concept would then be linked to the new merged Concept instead. They could be merged under one of the exisiting Concepts or a new Concept could be created and the old two removed. This method would invalidate existing queries using relevant Concepts & Terms.

In [ ]:
mdb.link_concepts_to_predicate("4jtzA3", "UWs6Di")

Created new Predicate with nanoid: isAxe4 and handle: exactMatch
Created has_subject relationship between source Predicate: isAxe4 and destination Concept: 4jtzA3
Created has_object relationship between source Predicate: isAxe4 and destination Concept: UWs6Di


In [ ]:
with driver.session() as session:
    id_1 = session.read_transaction(mdb.generate_unique_nanoid)
    session.write_transaction(mdb.create_concept, id_1)
    id_2 = session.read_transaction(mdb.generate_unique_nanoid)   
    session.write_transaction(mdb.create_concept, id_2)
driver.close()

Created new Concept with nanoid: emSArC
Created new Concept with nanoid: S0mbPX


In [ ]:
with driver.session() as session:
    term_id_1 = session.read_transaction(mdb.generate_unique_nanoid)
    session.write_transaction(mdb.create_term_from_nanoid, "Nelson", term_id_1)
    term_id_2 = session.read_transaction(mdb.generate_unique_nanoid)
    session.write_transaction(mdb.create_term_from_nanoid, "Nelly", term_id_2)
    term_id_3 = session.read_transaction(mdb.generate_unique_nanoid)
    session.write_transaction(mdb.create_term_from_nanoid, "Nelsinghouse", term_id_3)
    session.write_transaction(mdb.create_represents_relationship_from_nanoid, 
                                term_id_1, "emSArC")
    session.write_transaction(mdb.create_represents_relationship_from_nanoid, 
                                term_id_2, "emSArC")
    session.write_transaction(mdb.create_represents_relationship_from_nanoid, 
                                term_id_3, "S0mbPX")
    driver.close()

Created new Term with value: Nelson and nanoid: wZ1yrF
Created new Term with value: Nelly and nanoid: 6TntuU
Created new Term with value: Nelsinghouse and nanoid: klKsMN
Term with nanoid:wZ1yrF now represents Concept: emSArC
Term with nanoid:6TntuU now represents Concept: emSArC
Term with nanoid:klKsMN now represents Concept: S0mbPX


In [ ]:
with driver.session() as session:
    concept_1 = "emSArC"
    concept_2 = "S0mbPX"
    terms_1 = session.read_transaction(mdb.get_terms, concept_1)
    terms_2 = session.read_transaction(mdb.get_terms, concept_2)
    print(f"{terms_1}")
    print(f"{terms_2}")
driver.close()

['6TntuU', 'wZ1yrF']
['klKsMN']


In [ ]:
mdb.merge_two_concepts("emSArC", "S0mbPX")

Term with nanoid:klKsMN now represents Concept: emSArC
